In [ ]:
from time import time
import csv
from sklearn.cluster import k_means
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import svm
from sklearn import linear_model
from sklearn import ensemble
from sklearn import neighbors
from elm import ELMClassifier, ELMRegressor, GenELMClassifier, GenELMRegressor
from random_layer import RandomLayer, MLPRandomLayer, RBFRandomLayer, GRBFRandomLayer
from math import sqrt
import mysql as dbconn

if __name__ == "__main__":
db_conn = dbconn.MySQL()
sql='SELECT * FROM qdg1'
columns=db_conn.get_col_names(sql)


# with open('qdg_time.csv', 'w', newline='') as csvfile:
# 	spamwriter = csv.writer(csvfile, delimiter=',',
#                            quotechar=',', quoting=csv.QUOTE_MINIMAL)
# 	sql='SELECT * FROM qdg1'
# 	columns=db_conn.get_col_names(sql)
# 	spamwriter.writerow(columns)
# 	rs= db_conn.select(sql)
# 	# print(len(rs))
# 	for r in rs:
# 		spamwriter.writerow(r)

# with open("1.csv") as file:
# 	data = file.read()
# print(data)

data = pd.read_csv('qdg_time.csv')

shuchu = pd.read_csv('qdg_time_test.csv')

outputy=['qdg_l1y', 'qdg_l2y', 'qdg_l3y']

# outputy_else=['after_1mon', 'after_2mon', 'after_3mon', 'after_4mon', 'after_5mon', 'after_6mon']

columns_feature=columns.copy()

es = [
    # 'qdg_f10y','qdg_f11y'
    ]


for col in outputy:
    columns_feature.remove(col)
for col in es:
    columns_feature.remove(col)
# for col in els:
# 	columns_feature.remove(col)
# print('datadeliei',type(data))
X = data[columns_feature]
Y = data[outputy]
Y_array=np.array(Y)
# print(type(Y_array))
origal_mean=Y_array.mean(axis=0)#0表示列，1表示行
origal_std=Y_array.std(axis=0)
# print(origal_mean,origal_std)
data_scale=preprocessing.scale(np.array(data))
data_scale_df=pd.DataFrame(data_scale,columns=columns)

# shuchu
a1 = np.array(shuchu)
a2 = np.array(data)
mean=a2.mean(axis=0)#0表示列，1表示行
std=a2.std(axis=0)
# print(a1,origal_mean,origal_std)
row=a1.shape[0]
col=a1.shape[1]
# print(row,col)
for xi in range(row):
    for yi in range(col):

        a1[xi][yi]=(a1[xi][yi]-mean[yi])/std[yi]
        # print(a1[xi][yi])
a3=pd.DataFrame(a1,columns=columns)
x_shuchu = a3[columns_feature]
y_shuchu = a3[outputy]
y_shuchu = np.array(y_shuchu)
# prtin
# print(x_shuchu,y_shuchu)
# x_test1 = array([])
# shuchu

X_scale=data_scale_df[columns_feature]
Y_scale=data_scale_df[outputy]
x_train, x_test, y_train, y_test = train_test_split(X_scale,Y_scale,test_size=0.2,random_state=0)

# print('leixing',type(y_test))
# print(x_test,x_shuchu)

def try_different_method(clf):
    clf.fit(x_train,y_train)
    score = clf.score(x_test, y_test)
    result = clf.predict(x_test)
    result_shuchu = clf.predict(x_shuchu)
    return score,result,result_shuchu
svr = svm.SVR()
knn = neighbors.KNeighborsRegressor()
rf =ensemble.RandomForestRegressor(n_estimators=10)
linear_reg = linear_model.LinearRegression()
ada = ensemble.AdaBoostRegressor(n_estimators=50)
gbrt = ensemble.GradientBoostingRegressor(n_estimators=100)
score,y_predict,result_shuchu=try_different_method(rf)
y_test_1 = np.array(y_test)
# print(type(y_predict))
# print(type(y_test_1))
# print(result_shuchu)
for xi in range(1):
    for yi in range(3):

        # a1[xi][yi]=(a1[xi][yi]-mean[yi])/std[yi]y_shuchu
        result_shuchu[xi][yi]=result_shuchu[xi][yi]*origal_std[yi]+origal_mean[yi]
        y_shuchu[xi][yi]=y_shuchu[xi][yi]*origal_std[yi]+origal_mean[yi]
        print(result_shuchu[xi][yi],y_shuchu[xi][yi])

row_num=y_predict.shape[0]
col_num=y_predict.shape[1]
# print(row_num,col_num)
y_test_2 =  y_test_1.copy()
flag = 0
flag_num = 0
# print(row_num)
for xi in range(row_num):
    for yi in range(col_num):

        y_predict[xi][yi]=y_predict[xi][yi]*origal_std[yi]+origal_mean[yi]
        y_test_1[xi][yi]=y_test_1[xi][yi]*origal_std[yi]+origal_mean[yi]
        y_test_2[xi][yi] = y_test_1[xi][yi]
        # # print(y_test[xi][yi])
        y_test_1[xi][yi] = round(y_test_1[xi][yi], 1)

        y_test_1[xi][yi] = y_predict[xi][yi] - y_test_1[xi][yi] 
        y_test_1[xi][yi] = round(y_test_1[xi][yi], 1)

        cha = y_test_1[xi][yi]/y_predict[xi][yi]
        flag_num = flag_num+1
        if np.fabs(y_test_1[xi][yi])<10:
            flag = flag + 1

# '''rate 0f cha/predict<0.1'''
# print(y_test_1.shape[0])
print(flag_num,flag/72)

# print(type(y_test_1))
y_predict=y_predict[:,0]
y_test_2=y_test_2[:,0]
plt.figure()
plt.plot(np.arange(len(y_predict)),y_test_2,'go-',label='true value')
plt.plot(np.arange(len(y_predict)),y_predict,'ro-',label='predict value')
plt.title('score: %f'%score)
plt.legend()
plt.show()

